In [ ]:
import pandas as pd
import sqlite3
import os

print("Starting analysis...\n")

orders = pd.read_csv("orders.csv")
print("Orders loaded")

users = pd.read_json("users.json")
print("Users loaded")

if os.path.exists("restaurants.db"):
    os.remove("restaurants.db")

conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()
print("Restaurant database created")

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
print("Restaurants loaded")

step1 = orders.merge(users, on="user_id", how="left")
final_df = step1.merge(restaurants, on="restaurant_id", how="left")

final_df.to_csv("final_food_delivery_dataset.csv", index=False)
print("Final dataset created successfully")

print("\nMCQ 1: Gold member revenue by city")
gold_df = final_df[final_df["membership"] == "Gold"]
print(gold_df.groupby("city")["total_amount"].sum())

print("\nMCQ 2: Average order value by cuisine")
print(final_df.groupby("cuisine")["total_amount"].mean())

print("\nMCQ 3: Users spending more than 1000 total")
user_spend = final_df.groupby("user_id")["total_amount"].sum()
print(user_spend[user_spend > 1000].count())

print("\nMCQ 4: Revenue by restaurant rating range")
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
final_df["rating_range"] = pd.cut(final_df["rating"], bins=bins, labels=labels)
print(final_df.groupby("rating_range")["total_amount"].sum())

print("\nMCQ 5: Gold members average order value by city")
print(gold_df.groupby("city")["total_amount"].mean())

print("\nMCQ 6: Cuisine restaurant count vs revenue")
print(
    final_df.groupby("cuisine").agg(
        restaurants=("restaurant_id", "nunique"),
        revenue=("total_amount", "sum")
    )
)

print("\nMCQ 7: Percentage of orders by Gold members")
print(round((gold_df.shape[0] / final_df.shape[0]) * 100), "%")

print("\nMCQ 8: Restaurant with highest AOV but < 20 orders")
rest_stats = final_df.groupby("restaurant_name_y").agg(
    avg_order=("total_amount", "mean"),
    order_count=("order_id", "count")
)
print(
    rest_stats[rest_stats["order_count"] < 20]
    .sort_values(by="avg_order", ascending=False)
    .head(1)
)

print("\nMCQ 9: Revenue by membership + cuisine")
print(final_df.groupby(["membership", "cuisine"])["total_amount"].sum())

print("\nMCQ 10: Revenue by quarter")
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)
final_df["quarter"] = final_df["order_date"].dt.quarter
print(final_df.groupby("quarter")["total_amount"].sum())

print("\nNumerical Q1")
print(final_df[final_df["membership"] == "Gold"].shape[0])

print("\nNumerical Q2")
print(round(final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()))

print("\nNumerical Q3")
print(final_df["user_id"].nunique())

print("\nNumerical Q4")
print(round(final_df[final_df["membership"] == "Gold"]["total_amount"].mean(), 2))

print("\nNumerical Q5")
print(final_df[final_df["rating"] >= 4.5].shape[0])

print("\nNumerical Q6")
print(
    final_df[
        (final_df["membership"] == "Gold") &
        (final_df["city"] == "Chennai")
    ].shape[0]
)
